In [1]:
!pip install datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [11]:
import os, json, datasets
from bs4 import BeautifulSoup

In [12]:
!git clone https://github.com/BCHSI/philter-deidstable1_mirror.git ./philter/src
%cd ./philter/src
!git checkout v1.2024.1
!pip install -r requirements.txt
%cd ../..

fatal: destination path './philter/src' already exists and is not an empty directory.
/content/philter/src
HEAD is now at 38d0df55 fixing dynamic_protect_list log
/content


In [13]:
import nltk
nltk.download('averaged_perceptron_tagger', quiet=True)

True

In [ ]:
os.mkdir('./philter/results')

In [15]:
%rm -rf ./philter/results ./philter/data
os.makedirs('./philter/results')
os.makedirs('./philter/data')

In [16]:
from google.colab import drive
drive.mount('/content/drive')

# Collect testing data (without tags)
test_dataset = datasets.load_from_disk('drive/MyDrive/LiboMsc/processed_datasets/test_dataset')

for letter in test_dataset:
  with open(f"./philter/data/{letter['note_id']}.txt", 'w+') as f:
    f.write(letter['text'])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
%rm -rf ./gold_tags
os.mkdir('./gold_tags')

In [18]:
# HIPAA-PHI only tags
NOT_HIPAA_COMPOSED_TAGS = {
    "NAME": ["DOCTOR", "USERNAME"],
    "LOCATION": ["ROOM", "DEPARTMENT", "HOSPITAL", "STATE", "COUNTRY", "OTHER"],
    "CONTACT": ["URL", "IPADDRESS"],
    "AGE": [],
    "DATE": [],
    "ID": [],
}
NOT_HIPAA_SING_TAGS = ["PROFESSION"]
DEFAULT_TAGS = ['deIdi2b2', 'TEXT', 'TAGS']


# Remove all additional tags (that are not HIPAA-PHI mandatory)
def process_xml(input_folder, output_folder):
  for filename in os.listdir(input_folder):
    if not filename.endswith(".xml"):
      continue

    filepath = os.path.join(input_folder, filename)
    with open(filepath, 'r') as f:
      soup = BeautifulSoup(f, 'xml')

    for tag in soup.find_all():
      if tag.name not in DEFAULT_TAGS and (tag.name in NOT_HIPAA_SING_TAGS or tag['TYPE'] in NOT_HIPAA_COMPOSED_TAGS[tag.name]):
        tag.decompose()

    output_filepath = os.path.join(output_folder, filename)
    with open(output_filepath, 'w') as f:
      f.write(str(soup))

process_xml(input_folder="./drive/MyDrive/LiboMsc/data/i2b2_2024_T1_test", output_folder="./gold_tags")

In [19]:
# Evaluate accuracy of Philter on testing data against gold tags
%cd philter/src
!python3 deidpipe.py -i ../data/ -o ../results/ -f configs/philter_one2024.json -d False -e True -a ../../gold_tags
%cd ../..

/content/philter/src
read args
detecting PHI coordinates
Initializing Philter
Coordinates Identified
Pos_tags identified
detecting PHI types
normalizing PHI
../data/347-04.txt
../data/266-01.txt
../data/266-01.txt
../data/266-01.txt
../data/266-01.txt
../data/266-01.txt
../data/266-01.txt
../data/266-03.txt
../data/266-03.txt
../data/266-03.txt
../data/266-03.txt
../data/266-03.txt
../data/266-03.txt
../data/266-03.txt
../data/266-03.txt
../data/206-02.txt
../data/266-02.txt
../data/266-02.txt
../data/266-02.txt
../data/266-02.txt
../data/266-02.txt
../data/266-02.txt
../data/266-02.txt
../data/310-04.txt
../data/166-02.txt
../data/196-04.txt
../data/341-03.txt
../data/232-02.txt
../data/266-04.txt
../data/266-04.txt
../data/266-04.txt
../data/266-04.txt
../data/266-04.txt
../data/266-04.txt
../data/266-04.txt
../data/266-04.txt
../data/387-02.txt
../data/118-02.txt
../data/232-03.txt
../data/341-05.txt
../data/210-04.txt
../data/112-04.txt
../data/202-01.txt
transforming texts
saving 

In [20]:
with open('./philter/results/eval/summary.json', 'r') as f:
  eval_results = json.load(f)

eval_results

{'tp': 16645,
 'fp': 9969,
 'tn': 312880,
 'fn': 743,
 'precision': 0.6254227098519576,
 'recall': 0.9572693811824247,
 'retention': 0.9691217875849081,
 'ctp': 16546,
 'cfp': 10068,
 'ctn': 313052,
 'cfn': 571,
 'cprecision': 0.6217028631547306,
 'crecall': 0.9666413507039785,
 'cretention': 0.9688412973508295,
 'tp_dates_normalized': 11851,
 'tp_dates_obscured': 124,
 'fp_dates_normalized': 17,
 'fp_dates_obscured': 415,
 'fn_dates': 557}